<a href="https://colab.research.google.com/github/atanupay06/crop-yield-predictor/blob/main/yield_pred_model_v_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q joblib fastapi uvicorn nest-asyncio pyngrok streamlit requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.6 MB/s eta 0:00:00


In [90]:
#1- Library import
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import os


In [91]:
# 2. Load data
DATA_PATH = "/content/Crop Prediction dataset.csv"  # adjust if different
df = pd.read_csv(DATA_PATH)
df.head(), df.shape

(                    State_Name District_Name  Crop_Year       Season  \
 0  Andaman and Nicobar Islands      NICOBARS       2000  Kharif        
 1  Andaman and Nicobar Islands      NICOBARS       2000  Kharif        
 2  Andaman and Nicobar Islands      NICOBARS       2000  Kharif        
 3  Andaman and Nicobar Islands      NICOBARS       2000  Whole Year    
 4  Andaman and Nicobar Islands      NICOBARS       2000  Whole Year    
 
                   Crop  Temperature  Humidity  Soil_Moisture    Area  \
 0             Arecanut           36        35             45  1254.0   
 1  Other Kharif pulses           37        40             46     2.0   
 2                 Rice           36        41             50   102.0   
 3               Banana           37        42             55   176.0   
 4            Cashewnut           36        40             54   720.0   
 
    Production  
 0      2000.0  
 1         1.0  
 2       321.0  
 3       641.0  
 4       165.0  ,
 (49999, 10))

In [92]:
# 3~ Quick EDA and missing check
print(df.info())
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   State_Name     49999 non-null  object 
 1   District_Name  49999 non-null  object 
 2   Crop_Year      49999 non-null  int64  
 3   Season         49999 non-null  object 
 4   Crop           49999 non-null  object 
 5   Temperature    49999 non-null  int64  
 6   Humidity       49999 non-null  int64  
 7   Soil_Moisture  49999 non-null  int64  
 8   Area           49999 non-null  float64
 9   Production     49784 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 3.8+ MB
None
State_Name         0
District_Name      0
Crop_Year          0
Season             0
Crop               0
Temperature        0
Humidity           0
Soil_Moisture      0
Area               0
Production       215
dtype: int64


In [ ]:
df['Crop'].unique()

array(['Arecanut', 'Other Kharif pulses', 'Rice', 'Banana', 'Cashewnut',
       'Coconut ', 'Dry ginger', 'Sugarcane', 'Sweet potato', 'Tapioca',
       'Black pepper', 'Dry chillies', 'other oilseeds', 'Turmeric',
       'Maize', 'Moong(Green Gram)', 'Urad', 'Arhar/Tur', 'Groundnut',
       'Sunflower', 'Bajra', 'Castor seed', 'Cotton(lint)', 'Horse-gram',
       'Jowar', 'Korra', 'Ragi', 'Tobacco', 'Gram', 'Wheat', 'Masoor',
       'Sesamum', 'Linseed', 'Safflower', 'Onion', 'other misc. pulses',
       'Samai', 'Small millets', 'Coriander', 'Potato',
       'Other  Rabi pulses', 'Soyabean', 'Beans & Mutter(Vegetable)',
       'Bhindi', 'Brinjal', 'Citrus Fruit', 'Cucumber', 'Grapes', 'Mango',
       'Orange', 'other fibres', 'Other Fresh Fruits', 'Other Vegetables',
       'Papaya', 'Pome Fruit', 'Tomato', 'Rapeseed &Mustard', 'Mesta',
       'Cowpea(Lobia)', 'Lemon', 'Pome Granet', 'Sapota', 'Cabbage',
       'Peas  (vegetable)', 'Niger seed', 'Bottle Gourd', 'Sannhamp',
       'Va

In [ ]:
df['Season'].unique()
print(df['Season'].value_counts())

In [97]:
df['Season'] = df['Season'].str.strip()

In [ ]:
df[df['Production'].isnull()].shape

(215, 10)

In [93]:
#Dropping #N/A Production rows

df_clean = df.dropna(subset=['Production']).copy()

In [94]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49784 entries, 0 to 49998
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   State_Name     49784 non-null  object 
 1   District_Name  49784 non-null  object 
 2   Crop_Year      49784 non-null  int64  
 3   Season         49784 non-null  object 
 4   Crop           49784 non-null  object 
 5   Temperature    49784 non-null  int64  
 6   Humidity       49784 non-null  int64  
 7   Soil_Moisture  49784 non-null  int64  
 8   Area           49784 non-null  float64
 9   Production     49784 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 4.2+ MB


In [95]:
# Checking Area not null and >0
df_clean = df_clean.dropna(subset=['Area'])
df_clean = df_clean[df_clean['Area'] > 0]

In [98]:
df['Season'].unique()

array(['Kharif', 'Whole Year', 'Autumn', 'Rabi', 'Summer', 'Winter'],
      dtype=object)

In [ ]:
print("After dropping missing Production rows:", df_clean.shape)

After dropping missing Production rows: (49784, 10)


There are no Area <=0


In [96]:
#New column Yield
df_clean['Yield'] = df_clean['Production'] / df_clean['Area']
df_clean['Yield'].describe()

,Yield
count,49784.000000
mean,84.233237
std,929.707033
min,0.000000
25%,0.616852
50%,1.053023
75%,2.876939
max,33089.005477


In [ ]:
#Numerical and cateagorical feature selection (keeping year as categorical)
num_features = ['Temperature','Humidity','Soil_Moisture','Area']
cat_features = ['State_Name','District_Name','Crop_Year','Season','Crop']

In [ ]:
#Safe guard
for c in num_features:
    df_clean[c] = pd.to_numeric(df_clean[c], errors='coerce')
df_clean['Crop_Year'] = df_clean['Crop_Year'].astype(str)

# fill numerical missing values it may be null
df_clean[num_features] = df_clean[num_features].fillna(df_clean[num_features].median())


In [ ]:
#Training and testing splitting (20% test size)
X = df_clean[cat_features + num_features]
y = df_clean['Yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape)

(39827, 9) (9957, 9)


In [ ]:
# categorical encoder
cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# numeric scaler
num_transformer = StandardScaler()

preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, ['State_Name','District_Name','Crop_Year','Season','Crop'])
], remainder='drop')

In [ ]:
# Linear Regression pipeline
pipe_lr = Pipeline([
    ('preproc', preprocessor),
    ('model', LinearRegression())
])
pipe_lr.fit(X_train, y_train)

# Random Forest pipeline (basic)
pipe_rf = Pipeline([
    ('preproc', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])
pipe_rf.fit(X_train, y_train)


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Temperature', 'Humidity',
                                                   'Soil_Moisture', 'Area']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['State_Name',
                                                   'District_Name', 'Crop_Year',
                                                   'Season', 'Crop'])])),
                ('model', RandomForestRegressor(n_jobs=-1, random_state=42))])

In [99]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49784 entries, 0 to 49998
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   State_Name     49784 non-null  object 
 1   District_Name  49784 non-null  object 
 2   Crop_Year      49784 non-null  int64  
 3   Season         49784 non-null  object 
 4   Crop           49784 non-null  object 
 5   Temperature    49784 non-null  int64  
 6   Humidity       49784 non-null  int64  
 7   Soil_Moisture  49784 non-null  int64  
 8   Area           49784 non-null  float64
 9   Production     49784 non-null  float64
 10  Yield          49784 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 4.6+ MB


In [ ]:
#model evaluation
def eval_model(pipe, X_t, y_t, name="Model"):
    preds = pipe.predict(X_t)
    mae = mean_absolute_error(y_t, preds)
    mse = mean_squared_error(y_t, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_t, preds)
    print(f"{name} -> MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")
    return {'mae':mae,'rmse':rmse,'r2':r2}

print("LinearRegression:")
eval_model(pipe_lr, X_test, y_test, "LinearRegression")
print("RandomForest:")
eval_model(pipe_rf, X_test, y_test, "RandomForest")

LinearRegression:
LinearRegression -> MAE: 97.9848, RMSE: 581.9692, R2: 0.6229
RandomForest:
RandomForest -> MAE: 25.6279, RMSE: 347.8464, R2: 0.8653


{'mae': 25.627894443344776,
 'rmse': np.float64(347.84638562183244),
 'r2': 0.8652673909575807}

In [ ]:
# Colab UI: copy-paste this entire cell and run
# If ipywidgets not installed, uncomment next line
# !pip install -q ipywidgets

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import joblib
import pandas as pd
import numpy as np
import os

# ======= CONFIGURE PATH TO YOUR SAVED MODEL ========
MODEL_PATH = "/content/artifacts/yield_rf_pipeline.joblib"   # change if needed
# If you saved meta or training df medians, set those paths too (optional)
DF_CLEAN_PATH = "/content/df_clean_sample.parquet"  # optional: if you saved training sample
# ===================================================

# Try load model
model = None
model_load_err = None
if os.path.exists(MODEL_PATH):
    try:
        model = joblib.load(MODEL_PATH)
    except Exception as e:
        model_load_err = str(e)
else:
    model_load_err = f"Model file not found at {MODEL_PATH}. Please upload or change MODEL_PATH."

# Try load df_clean if exists (for medians fallback)
df_clean = None
if os.path.exists(DF_CLEAN_PATH):
    try:
        df_clean = pd.read_parquet(DF_CLEAN_PATH)
    except Exception:
        try:
            df_clean = pd.read_csv(DF_CLEAN_PATH)
        except Exception:
            df_clean = None

# Utility: get median defaults (if df_clean available)
def get_default_medians():
    if df_clean is not None:
        return {
            'Temperature': float(df_clean['Temperature'].median()),
            'Humidity': float(df_clean['Humidity'].median()),
            'Soil_Moisture': float(df_clean['Soil_Moisture'].median()),
            'Area': float(df_clean['Area'].median())
        }
    else:
        # fallback fixed defaults
        return {'Temperature': 30.0, 'Humidity': 50.0, 'Soil_Moisture': 40.0, 'Area': 100.0}

default_meds = get_default_medians()

# Create widgets
district_w = widgets.Text(value='NICOBARS', description='District:', layout=widgets.Layout(width='60%'))
year_w = widgets.Text(value='2000', description='Year:')
crop_w = widgets.Text(value='Coconut', description='Crop:', layout=widgets.Layout(width='60%'))

temp_w = widgets.FloatText(value=default_meds['Temperature'], description='Temp (°C):')
hum_w = widgets.FloatText(value=default_meds['Humidity'], description='Humidity (%):')
soil_w = widgets.FloatText(value=default_meds['Soil_Moisture'], description='Soil Moist:')
area_w = widgets.FloatText(value=default_meds['Area'], description='Area:')

prod_w = widgets.FloatText(value=np.nan, description='Production (opt):')
season_w = widgets.Text(value='Whole Year', description='Season:')
state_w = widgets.Text(value='', description='State (opt):')

predict_btn = widgets.Button(description='Predict', button_style='success')
out = widgets.Output(layout={'border': '1px solid gray'})

# Function to run on click
def on_predict_clicked(b):
    with out:
        clear_output()
        # Check model
        if model is None:
            print("Model not loaded. Error:", model_load_err)
            print("Upload your model to MODEL_PATH or change MODEL_PATH variable.")
            return
        # build input row
        try:
            input_row = {
                'State_Name': state_w.value if state_w.value else "Unknown",
                'District_Name': district_w.value,
                'Crop_Year': str(year_w.value),
                'Season': season_w.value if season_w.value else "Whole Year",
                'Crop': crop_w.value,
                'Temperature': float(temp_w.value),
                'Humidity': float(hum_w.value),
                'Soil_Moisture': float(soil_w.value),
                'Area': float(area_w.value) if not np.isnan(area_w.value) else default_meds['Area']
            }
        except Exception as e:
            print("Invalid numeric input:", e)
            return

        input_df = pd.DataFrame([input_row])
        # Predict
        try:
            pred_yield = float(model.predict(input_df)[0])
        except Exception as e:
            print("Prediction error:", e)
            return

        pred_production = pred_yield * input_row['Area']
        print("=== Prediction result ===")
        print(f"Predicted Yield (production per unit area): {pred_yield:.4f}")
        print(f"Predicted Production (for area {input_row['Area']}): {pred_production:.4f}")
        # If user provided actual production, compare
        if not (prod_w.value is None or np.isnan(prod_w.value)):
            actual_prod = float(prod_w.value)
            actual_yield = actual_prod / input_row['Area'] if input_row['Area'] != 0 else np.nan
            abs_err = abs(actual_yield - pred_yield)
            pct_err = (abs_err / actual_yield * 100) if actual_yield != 0 and not np.isnan(actual_yield) else np.nan
            print("\n=== Actual vs Predicted ===")
            print(f"Actual Production: {actual_prod}")
            print(f"Actual Yield: {actual_yield:.4f}")
            print(f"Absolute Error in Yield: {abs_err:.4f}")
            if not np.isnan(pct_err):
                print(f"Percentage Error: {pct_err:.2f}%")
        # Optionally show used features
        print("\nUsed features:")
        for k,v in input_row.items():
            print(f"  {k}: {v}")

predict_btn.on_click(on_predict_clicked)

# Layout
left_col = widgets.VBox([district_w, crop_w, year_w, season_w, state_w])
right_col = widgets.VBox([temp_w, hum_w, soil_w, area_w, prod_w, predict_btn])
ui = widgets.HBox([left_col, right_col])

display(HTML("<h3>Crop Yield Quick Check UI (Colab)</h3>"))
display(ui)
display(out)

# Instructions
display(HTML("""
<b>Instructions:</b>
<ul>
<li>If your saved model is not in <code>/content/artifacts/yield_rf_pipeline.joblib</code>, change the <code>MODEL_PATH</code> variable at the top of this cell.</li>
<li>Fill District/Year/Crop and optional numeric overrides. If you provide Production, the UI will compute actual yield and show error vs predicted.</li>
<li>If you haven't saved a model yet, run the training cell first and save model via <code>joblib.dump(pipe, 'artifacts/yield_rf_pipeline.joblib')</code>.</li>
</ul>
"""))


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Output(layout=Layout(border='1px solid gray'))

In [100]:
import os

# Define the directory to save the cleaned data
output_dir = "/content/artifacts"
os.makedirs(output_dir, exist_ok=True)

# Path for the Parquet file (efficient for data storage and retrieval)
parquet_output_path = os.path.join(output_dir, "df_clean.parquet")
df_clean.to_parquet(parquet_output_path, index=False)

# Path for the CSV file (widely compatible)
csv_output_path = os.path.join(output_dir, "df_clean.csv")
df_clean.to_csv(csv_output_path, index=False)

print(f"df_clean exported to: {parquet_output_path}")
print(f"df_clean exported to: {csv_output_path}")

df_clean exported to: /content/artifacts/df_clean.parquet
df_clean exported to: /content/artifacts/df_clean.csv


The `df_clean` dataset has been successfully exported to `df_clean.parquet` and `df_clean.csv` within the `/content/artifacts/` directory. You can find these files in the Colab file browser.